# Example Notebook for tweaked BCS Module

There is already a built-in drudge module for `ReducedBCSDrudge`, but this tweak helps deal with general indices (instead of the original particle-hole based module), and has an additional function to evaluate AGP expectation values.

### 1. Start by importing the modules as usual

In [1]:
# from pyspark import SparkContext
from dummy_spark import SparkContext
from sympy import *
from drudge import *
from bcs import * # This is the tweaked module -- bcs.py

ctx = SparkContext()
dr = ReducedBCSDrudge(ctx)

# Getting the namespace
names = dr.names
Pdag = names.Pdag
P = names.P
N = names.N
p,q,r,s = dr.all_orb_dumms[:4]

### 2. Define the various AGP-RDM's with their appropriate symmetry

Note that this list can be large or small based on your application or needs.

In [3]:
#==================================== 
# AGP expected values:
#====================================
# case: z00 
Z00 = IndexedBase('Z00')

# case: z02 
Z02 = IndexedBase('Z02')
dr.set_symm(Z02,
    Perm([1,0],IDENT),
    valence=2
)

# case: z04
Z04 = IndexedBase('Z04')
dr.set_symm(Z04,
            Perm([1,0,2,3],IDENT),
            Perm([0,1,3,2],IDENT),
            Perm([2,3,0,1],IDENT),
            valence = 4
)

# case: z06
Z06 = IndexedBase('Z06')
dr.set_symm(Z06,
            Perm([1,0,2,3,4,5],IDENT),
            Perm([0,2,1,3,4,5],IDENT),
            Perm([0,1,2,4,3,5],IDENT),
            Perm([0,1,2,3,5,4],IDENT),
            Perm([3,4,5,0,1,2],IDENT),
            valence=6
)

# case: z08
Z08 = IndexedBase('Z08')
dr.set_symm(Z08,
            Perm([1,0,2,3,4,5,6,7],IDENT),
            Perm([0,2,1,3,4,5,6,7],IDENT),
            Perm([0,1,3,2,4,5,6,7],IDENT),
            Perm([0,1,2,3,5,4,6,7],IDENT),
            Perm([0,1,2,3,4,6,5,7],IDENT),
            Perm([0,1,2,3,4,5,7,6],IDENT),
            Perm([4,5,6,7,0,1,2,3],IDENT),
            valence=8
)

#--------------------------------------
# case: z11
Z11 = IndexedBase('Z11')

# case: z13
Z13 = IndexedBase('Z13')
dr.set_symm(Z13,
            Perm([2,1,0],IDENT),
            valence=3
) 

# case: z15
Z15 = IndexedBase('Z15')
dr.set_symm(Z15,
            Perm([1,0,2,3,4],IDENT),
            Perm([0,1,2,4,3],IDENT),
            Perm([3,4,2,0,1],IDENT),
            valence=5
)

# case: z17
Z17 = IndexedBase('Z17')
dr.set_symm(Z17,
            Perm([1,0,2,3,4,5,6],IDENT),
            Perm([0,2,1,3,4,5,6],IDENT),
            Perm([0,1,2,3,5,4,6],IDENT),
            Perm([0,1,2,3,4,6,5],IDENT),
            Perm([4,5,6,3,0,1,2],IDENT),
            valence=7
)

#--------------------------------------
# case: z22
Z22 = IndexedBase('Z22')
dr.set_symm(Z22,
            Perm([1,0],IDENT),
            valence=2
) 

# case: z24
Z24 = IndexedBase('Z24')
dr.set_symm(Z24,
            Perm([3,1,2,0],IDENT),
            Perm([0,2,1,3],IDENT),
            valence=4
) 

# case: z26
Z26 = IndexedBase('Z26')
dr.set_symm(Z26,
            Perm([0,1,3,2,4,5],IDENT),
            Perm([1,0,2,3,4,5],IDENT),
            Perm([0,1,2,3,5,4],IDENT),
            Perm([4,5,2,3,0,1],IDENT),
            valence=6
) 

#--------------------------------------
#case:z33
Z33 = IndexedBase('Z33')
dr.set_symm(Z33,
            Perm([1,0,2],IDENT),
            Perm([0,2,1],IDENT),
            valence=3
)

# case: z35
Z35 = IndexedBase('Z35')
dr.set_symm(Z35,
            Perm([0,2,1,3,4],IDENT),
            Perm([0,1,3,2,4],IDENT),
            Perm([4,1,2,3,0],IDENT),
            valence=5
) 

#--------------------------------------
# case: z44
Z44 = IndexedBase('Z44')
dr.set_symm(Z44,
            Perm([1,0,2,3],IDENT),
            Perm([0,2,1,3],IDENT),
            Perm([0,1,3,2],IDENT),
            valence=4
) 

Zlist = [
    [Z00, Z02, Z04, Z06, Z08], 
    [Z11, Z13, Z15, Z17], 
    [Z22, Z24, Z26], 
    [Z33, Z35], 
    [Z44]
]

### 3. Ready to Define expressions and evaluate expectations

In this example, we define the Hamiltonian

In [4]:
eps = IndexedBase('e')
G = IndexedBase('G')

# Impose symmetry on G
dr.set_symm(G,
    Perm([1,0],IDENT),
    valence=2
)

# Define H:
H2 = - dr.sum((p,  names.A), (q,  names.A), G[p,q]*Pdag[p]*P[q])
H1 = dr.simplify( dr.einst(eps[p]*N[p] ) )
H = H1 + H2

Display the Hamiltonian

In [5]:
H.simplify().display()

<IPython.core.display.Math object>

Evaluate the expectation value over AGP. The function `eval_agp()` does this task -- it takes two inputs:
* the expression
* the list of RDM's (`Zlist`) that we defined above
and returns the expectation value as a symbolic expression in terms of RDM's

In [6]:
agp_energy = dr.eval_agp(H,Zlist)
agp_energy = dr.simplify( agp_energy )
agp_energy.display()

<IPython.core.display.Math object>